# Задача

В рамках данной задачи предстоит научиться получать наилучшее приближение $\sigma\in\mathbb{C}^n$ истинного квантового состояния $\rho\in\mathbb{C}^n$ по некоторой обучающей выборке $X_{train}=\{E_m,y_m\}_{m=0}^{N}$, где $E_m$ - некоторая матрица (называемая оператор, проектор), $y_m$ - некоторое наблюдение (проекция истинного состояния $\rho$ на этот оператор). 

В качестве оптимизируемого функционала предлагается минимизировать следующую функцию:
$$min_\sigma[f(\sigma)=\Sigma_m(Tr(E_m\sigma)-Tr(E_m\rho))^2=\Sigma_m(Tr(E_m\sigma)-y_m)^2]$$
$$\sigma\geq0,Tr(\sigma)=1$$
гдe $\sigma\geq0$ означает положительную полуопределённость обучаемой матрицы $\sigma$ 

# Решение

Поскольку матрица $\sigma$ - положительно полуопределённая, сделаем разложение Холецкого: $\sigma=M^*M$. 

Тогда 
$$Tr(\sigma)=Tr(M^*M)=\Sigma_{a_m\in M} a_m^*a_m=\Sigma_{a_m\in M} |a_m|^2$$

Поскольку для любой положительно полуопределённой матрицы существует разложение Холецкого, и, наоборот, для любой матрицы $M\in\mathbb{C}^n$ матрица $M^*M$ является положительно полуопределённой, мы можем переписать оптимизационную задачу в виде

$$min_M[f(M)=\Sigma_m(Tr(E_mM^*M)-y_m)^2]$$
$$Tr(M^*M)=\Sigma_{a_m\in M}|a_m|^2=1$$

Эту задачу можно было бы решать обычным методом множителей Лагранжа, но можно просто перенормировать $M$ на $\sqrt{Tr(M^*M)}$. Тогда задача сведётся к задаче безусловной оптимизации

$$min_M[f(M)=\Sigma_m(Tr(E_m\frac{M^*M}{Tr(M^*M)})-y_m)^2=\Sigma_m(Tr(E_m\frac{M^*M}{\Sigma_{a_m\in M}|a_m|^2})-y_m)^2]$$

# Код

## Генерация датасета

In [1]:
import sys
import torch
import numpy as np
sys.path.append('..')
from lib import simulator
from lib.pytorch_complex_tensor import ComplexTensor

In [2]:
rho = np.array([[1., 0.], [0., 0.]]) # target state
dim = rho.shape[0]
projectors_cnt = 10
measurements_cnt = 100
train_size = projectors_cnt * measurements_cnt
train_X, train_y = simulator.generate_dataset(rho, projectors_cnt, measurements_cnt)

## Обучение модели

In [3]:
M = ComplexTensor(simulator.randomMixedState(rho.shape[0]))
M.requires_grad = True
opt = torch.optim.SGD([M], lr=1e-1)
epoches = 1000

def trace(tensor):
    return sum([tensor[i:i+1,i:i+1] for i in range(tensor.shape[0])])

In [4]:
for epoch in range(epoches):
    sigma = M.t(conjugate=True).mm(M)
    norm = trace(sigma).real.sum()

    loss = 0
    for E_m,y_m in zip(train_X, train_y.astype('float64')):
        E_m = ComplexTensor(E_m)
        loss += ((trace(E_m.mm(sigma)/norm)-y_m)**2).sum()
    loss /= train_size
    loss.backward()
    opt.step()
    opt.zero_grad()
    print(f'Epoch: {epoch}, Loss: {loss.detach().cpu().numpy()}')

Epoch: 0, Loss: 0.5431708693504333
Epoch: 1, Loss: 0.517189085483551
Epoch: 2, Loss: 0.48283445835113525
Epoch: 3, Loss: 0.4408557415008545
Epoch: 4, Loss: 0.3942459523677826
Epoch: 5, Loss: 0.3476411998271942
Epoch: 6, Loss: 0.3054879903793335
Epoch: 7, Loss: 0.2703815996646881
Epoch: 8, Loss: 0.24276570975780487
Epoch: 9, Loss: 0.22171346843242645
Epoch: 10, Loss: 0.20585192739963531
Epoch: 11, Loss: 0.19387571513652802
Epoch: 12, Loss: 0.1847461313009262
Epoch: 13, Loss: 0.17769379913806915
Epoch: 14, Loss: 0.17216533422470093
Epoch: 15, Loss: 0.1677687019109726
Epoch: 16, Loss: 0.16422514617443085
Epoch: 17, Loss: 0.16133145987987518
Epoch: 18, Loss: 0.15894284844398499
Epoch: 19, Loss: 0.15695065259933472
Epoch: 20, Loss: 0.15527288615703583
Epoch: 22, Loss: 0.1526326686143875
Epoch: 23, Loss: 0.15158408880233765
Epoch: 24, Loss: 0.15067677199840546
Epoch: 25, Loss: 0.1498861163854599
Epoch: 26, Loss: 0.1491931974887848
Epoch: 27, Loss: 0.14858292043209076
Epoch: 28, Loss: 0.14804

KeyboardInterrupt: 

In [5]:
sigma = sigma/norm
sigma = sigma.detach().numpy()[:dim] + 1j*sigma.detach().numpy()[dim:]
sigma

array([[0.98129565+0.j        , 0.00325902-0.00831092j],
       [0.00325902+0.00831092j, 0.01870438+0.j        ]], dtype=complex64)

In [6]:
simulator.isDensityMatrix(sigma)

True